#### https://blog.paperspace.com/object-localization-using-pytorch-1/

In [1]:
import importlib as impL
import tutorial01 as t01
from helperFuncs import is_cuda_available, check_and_return_device
import localization_example_helper_funcs as t01_hf
is_cuda_available()
device = check_and_return_device()

Cuda is available
cuda


In [2]:
#labels, boxes, img_list = t01.load_data()
#dataloader, valdataloader = t01.preprocess_data(img_list, boxes, labels)

### model1, acc_list1 = t01_hf.train(dataloader, valdataloader,  model=None, num_of_epochs=50, start_from_scratch=True)

In [3]:
impL.reload(t01_hf)
model_sample = t01_hf.SampleNetwork()
model_sample

SampleNetwork(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(24, 48, kernel_size=(5, 5), stride=(1, 1))
  (conv5): Conv2d(48, 192, kernel_size=(5, 5), stride=(1, 1))
  (class_fc1): Linear(in_features=1728, out_features=240, bias=True)
  (class_out): Linear(in_features=240, out_features=2, bias=True)
  (box_fc1): Linear(in_features=1728, out_features=240, bias=True)
  (box_out): Linear(in_features=240, out_features=4, bias=True)
)

In [10]:
import torchVisExam02 as tv02
impL.reload(tv02)
img1, img2 = tv02.run_02(device)
backbone = model_sample
backbone.out_channels = 240
model_rc = tv02.run_03(backbone)

torch.Size([3, 256, 256])


In [11]:
model_rc.eval()
model_rc = model_rc.to(device)

In [12]:
import torch 
print(torch.Tensor.size(img1))
print(torch.Tensor.size(img2))
predictions = model_rc([img1, img2])

torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
size of *1.input tensor = torch.Size([2, 3, 256, 256])
size of *1.after maxpool tensor = torch.Size([2, 6, 126, 126])
size of *2.after maxpool tensor = torch.Size([2, 12, 61, 61])
size of *3.after maxpool tensor = torch.Size([2, 24, 28, 28])
size of *4.after maxpool tensor = torch.Size([2, 48, 12, 12])
size of *5.after avgpool tensor = torch.Size([2, 192, 3, 3])
size of *6.after flatten tensor = torch.Size([2, 1728])
size of *4.1.class_t = torch.Size([2, 2])
size of *4.2.box_t = torch.Size([2, 4])


/home/doga/anaconda3/envs/objectLocalization/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


AttributeError: 'list' object has no attribute 'values'

In [ ]:
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import MultiScaleRoIAlign

In [ ]:
backbone = model1
backbone.out_channels = 120
anchor_generator = AnchorGenerator(sizes=((32, 64, 128),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
mask_roi_pooler = MultiScaleRoIAlign(featmap_names=['0'], output_size=14, sampling_ratio=2)

In [ ]:
from torchvision.models.detection import MaskRCNN
model_rc = MaskRCNN(backbone,
                 num_classes=2, rpn_anchor_generator=anchor_generator,
                 box_roi_pool=roi_pooler, mask_roi_pool=mask_roi_pooler)
model_rc.eval()

In [ ]:
import torch 
import cv2
# x = [torch.rand(3, 300, 400).to(device), torch.rand(3, 500, 400).to(device)]
def im2model(img_path, device):
    img = cv2.imread(img_path)
    img = t01_hf.preprocess(img)
    img = torch.from_numpy(img).permute((0, 3, 1, 2)).float().to(device)
    return torch.squeeze(img)

In [ ]:
rand_img_path1 = t01_hf.get_random_img()
rand_img_path2 = t01_hf.get_random_img()
img1 = im2model(rand_img_path1, device)
img2 = im2model(rand_img_path2, device)
print(torch.Tensor.size(img1))

In [ ]:
predictions = model_rc([img1, img2])